In [66]:
import json
import os
import pandas as pd
import numpy as np

array = np.random.rand(3)
array[0]

0.2135636534288361

In [67]:
series = pd.Series(array)
series[0]

0.2135636534288361

In [68]:
series = pd.Series(array, index=['First', 'Second', 'Third'])
series['First']

0.2135636534288361

In [69]:
array2d = np.random.rand(3,2)
array2d[0, 1]

0.21676208405629094

In [70]:
df = pd.DataFrame(array2d)
try:
    df[0, 1]
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

KeyError((0, 1)) 
 context:  KeyError((0, 1))


In [71]:
df.columns = ['First', 'Second']
df

,First,Second
0,0.248769,0.216762
1,0.128458,0.555655
2,0.715982,0.268392


In [72]:
df['Second']

0    0.216762
1    0.555655
2    0.268392
Name: Second, dtype: float64

## <center><font color=tomato>reading from csv files</font></center>

In [73]:
df = pd.read_csv('data/artwork_data.csv', nrows=5, index_col='id')

In [74]:
COLS_TO_USE = ['id', 'artist', 'title', 'medium', 'year', 'acquisitionYear', 'height', 'width', 'units']
df = pd.read_csv('data/artwork_data.csv', index_col='id', usecols=COLS_TO_USE)

/Users/jalejo/.conda/envs/PythonLibraries/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


If we only want to use it in the Python scripts the simplest way is to use pickle, which is a native Python format for serialization.

In [75]:
df.to_pickle('data/dataframe.pickle')

In [76]:
df = pd.read_pickle('data/dataframe.pickle')

## <center><font color=tomato>reading json files</font></center>

In [77]:
records = [('Expresso', '5$'),
           ('Flat White', '10$')]

pd.DataFrame.from_records(records, columns=['Coffee', 'Price'])

,Coffee,Price
0,Expresso,5$
1,Flat White,10$


In [78]:
KEYS_TO_USE = ['id', 'all_artists', 'title', 'medium', 'acquisitionYear', 'height', 'width', 'units']

def get_record_from_file(file_path, keys_to_use):
    """
    Process single json file and return a tuple containing specific fields
    """
    with open(file_path) as artwork_file:
        content = json.load(artwork_file)

    record = []
    for field in keys_to_use:
        record.append(content[field])

    return tuple(record)

SAMPLE_JSON = 'data/artworks/a/000/a00001-1035.json'
SAMPLE_JSON = os.path.join('data', 'artworks', 'a', '000', 'a00001-1035.json')

get_record_from_file(SAMPLE_JSON, KEYS_TO_USE)

(1035,
 'Robert Blake',
 'A Figure Bowing before a Seated Old Man with his Arm Outstretched in Benediction. Verso: Indecipherable Sketch',
 'Watercolour, ink, chalk and graphite on paper. Verso: graphite on paper',
 1922,
 '419',
 '394',
 'mm')

In [79]:
def get_records_from_json(keys_to_use):
    """
    Traverse the directories with JSON files.
    For the first file in each directory call function for processing single file and go to the next directory
    """
    JSON_ROOT = os.path.join('data', 'artworks')
    artworks = []

    for root, _, files in os.walk(JSON_ROOT):
        for f in files:
            if f.endswith('json'):
                try:
                    record = get_record_from_file(os.path.join(root, f), keys_to_use)
                except Exception as e:
                    print(f"record in file {os.path.join(root, f)} not written, ",e.__repr__())
                artworks.append(record)
    df = pd.DataFrame.from_records(artworks, columns=keys_to_use, index='id')
    return df

df = get_records_from_json(KEYS_TO_USE)

record in file data/artworks/t/079/t07932-75078.json not written,  KeyError('acquisitionYear')
record in file data/artworks/t/077/t07706-78.json not written,  KeyError('acquisitionYear')
record in file data/artworks/t/118/t11879-83515.json not written,  KeyError('acquisitionYear')
record in file data/artworks/t/118/t11869-83517.json not written,  KeyError('acquisitionYear')
record in file data/artworks/t/118/t11872-83507.json not written,  KeyError('acquisitionYear')
record in file data/artworks/t/118/t11881-83516.json not written,  KeyError('acquisitionYear')
record in file data/artworks/t/118/t11880-83514.json not written,  KeyError('acquisitionYear')
record in file data/artworks/d/415/d41539-121203.json not written,  KeyError('acquisitionYear')
record in file data/artworks/d/415/d41537-120309.json not written,  KeyError('acquisitionYear')
record in file data/artworks/d/415/d41515-108224.json not written,  KeyError('acquisitionYear')
record in file data/artworks/d/415/d41519-111177.j

## <center><font color=tomato>basics</font></center>

In [80]:
indexes = list(df.index)

In [81]:
df.all_artists

id
6620          William Hogarth
5837              Thomas Hand
7843     Charles Samuel Keene
7833     Charles Samuel Keene
1586         Ford Madox Brown
                 ...         
20896             John Walker
20872               Juan Gris
20860           Pablo Picasso
20852           Pablo Picasso
20874               Juan Gris
Name: all_artists, Length: 69202, dtype: object

In [82]:
artists = df['all_artists']
len(pd.unique(artists))

3503

## <center><font color=tomato>filtering</font></center>

In [83]:
s = df['all_artists'] == 'George Jones'
s.value_counts()

False    68156
True      1046
Name: all_artists, dtype: int64

In [84]:
artists_counts = df['all_artists'].value_counts()
artists_counts['George Jones']

1046

In [85]:
df.columns

Index(['all_artists', 'title', 'medium', 'acquisitionYear', 'height', 'width',
       'units'],
      dtype='object')

In [86]:
df.columns = ['artist', 'title', 'medium', 'acquisitionYear', 'height', 'width',
       'units']

In [87]:
df.head(2)


,artist,title,medium,acquisitionYear,height,width,units
id,,,,,,,
6620,William Hogarth,A Scene from ‘The Beggar’s Opera’ VI,Oil paint on canvas,1909,762,572,mm
5837,Thomas Hand,Cottage and Hilly Landscape,Oil paint on canvas,1909,908,711,mm


## <center><font color=tomato>indexation</font></center>

In [126]:
df['artist'].name

'artist'

In [107]:
df.loc[1035, 'artist']

'Robert Blake'

In [25]:
df.iloc[0, 0]

'William Hogarth'

In [26]:
df.iloc[0, :]

artist                                  William Hogarth
title              A Scene from ‘The Beggar’s Opera’ VI
medium                              Oil paint on canvas
acquisitionYear                                    1909
height                                              762
width                                               572
units                                                mm
Name: 6620, dtype: object

In [27]:
df.iloc[0:2, 0:3]

,artist,title,medium
id,,,
6620,William Hogarth,A Scene from ‘The Beggar’s Opera’ VI,Oil paint on canvas
5837,Thomas Hand,Cottage and Hilly Landscape,Oil paint on canvas


In [28]:
try:
    df['height'] * df['width']
except Exception as e:
    print(e.__repr__())

TypeError("can't multiply sequence by non-int of type 'str'")


In [29]:
# There are some rubbish in the data
df['width'].sort_values().tail()

id
97852        998
44381        999
26521     ea.316
100089      each
96387       each
Name: width, dtype: object

In [30]:
# Try to convert
try:
    pd.to_numeric(df['width'])
except Exception as e:
    print(e.__repr__(), '\n context: ', e.__context__.__repr__())

ValueError('Unable to parse string "(diameter):" at position 3974') 
 context:  ValueError('Unable to parse string "(diameter):"')


In [31]:
# Force NANs (Not A Number)
pd.to_numeric(df['width'], errors='coerce')

id
6620     572.0
5837     711.0
7843       NaN
7833       NaN
1586     114.0
         ...  
20896    125.0
20872    320.0
20860    645.0
20852    688.0
20874    360.0
Name: width, Length: 69202, dtype: float64

In [32]:
df.loc[:, 'width'] = pd.to_numeric(df['width'], errors='coerce')
df.loc[:, 'height'] = pd.to_numeric(df['height'], errors='coerce')

In [33]:
df['height'] * df['width']

id
6620     435864.0
5837     645588.0
7843          NaN
7833          NaN
1586      20292.0
           ...   
20896     12375.0
20872     73600.0
20860    343785.0
20852    351568.0
20874     86400.0
Length: 69202, dtype: float64

In [34]:
# Assign = create new columns with size
area = df['height'] * df['width']
df = df.assign(area=area)

In [35]:
df['area'].max()

132462000.0

In [36]:
df['area'].idxmax()

98367

In [37]:
df.loc[df['area'].idxmax(), :]

artist                                Robert Therrien
title                No Title (Table and Four Chairs)
medium             Aluminium, steel, wood and plastic
acquisitionYear                                  2008
height                                          14850
width                                            8920
units                                              mm
area                                      1.32462e+08
Name: 98367, dtype: object

## <center><font color=tomato>Operations</font></center>
### <font color=lightGreen>iterating over groups</font>

In [38]:
grouped = df.groupby('artist')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [39]:
i = 2
for name, group_df in grouped:
    print(name, '\n', len(name) * '-', '\n', group_df)
    i -= 1
    if i == 0:
        break

?British School 
 --------------- 
                artist                                              title  \
id                                                                         
6652  ?British School  Portrait of a Gentleman, probably of the West ...   

                medium  acquisitionYear  height   width units       area  
id                                                                        
6652  Oil paint on oak             1927   785.0  1330.0    mm  1044050.0  
A. Belloguet 
 ------------ 
              artist                   title                     medium  \
id                                                                       
25065  A. Belloguet  Design for Music Cover  Graphite and ink on paper   
25069  A. Belloguet  Design for Music Cover  Graphite and ink on paper   

       acquisitionYear  height  width units     area  
id                                                    
25065             1997   220.0  279.0    mm  61380.0  
25069             

### <font color=lightGreen>filter</font>

In [40]:
grouped_titles = df.groupby('title')
title_counts = grouped_titles.size().sort_values(ascending=False)

condition = lambda x: len(x.index) > 1
dup_titles_df = grouped_titles.filter(condition)
dup_titles_df.sort_values('title', inplace=True)
dup_titles_df

,artist,title,medium,acquisitionYear,height,width,units,area
id,,,,,,,,
21345,Carroll Dunham,#1,Drypoint on paper,2002,579.0,392.0,mm,226968.0
20571,Sean Scully,#1,Aquatint on paper,1992,174.0,117.0,mm,20358.0
20572,Sean Scully,#2,Aquatint on paper,1992,278.0,323.0,mm,89794.0
21346,Carroll Dunham,#2,Drypoint on paper,2002,579.0,392.0,mm,226968.0
20573,Sean Scully,#3,Aquatint on paper,1992,260.0,319.0,mm,82940.0
...,...,...,...,...,...,...,...,...
97431,Andy Warhol,“The Runaway Pigeon”,Ink on paper on ink on paper,2009,205.0,280.0,mm,57400.0
97409,Andy Warhol,“The Runaway Pigeon”,"Ink, watercolour and graphite on paper",2009,237.0,272.0,mm,64464.0
97408,Andy Warhol,“The Runaway Pigeon”,"Ink, watercolour and graphite on paper",2009,295.0,269.0,mm,79355.0


## <center><font color=tomato>Outputting pandas</font></center>

In [41]:
df.to_excel('dataframeExcel.xlsx')